In [1]:
import numpy as np
import pandas as pd
import re
#my_regex = "\（.*?\）|\(.*?\)+\(\^一二三四五六七八九零十1234567890\)\]"
my_regex = r"[\(\（]([^一二三四五六七八九零十1234567890]{1}[^\)\）]+|[一二三四五六七八九零十1234567890]{1}[^\)\）]{1,})[\)\）]"

In [2]:
df=pd.read_csv('.//data//Result_TFIDF.csv', index_col=0)
df.head(2)

,court,date,no,sys,reason,type,mainText,judgement,Situation
id,,,,,,,,,
0,臺灣桃園地方法院,2018-04-18T00:00:00+08:00,"106,易,1093",刑事,妨害公務,判決,邱建豪犯妨害公務執行罪，累犯，處拘役拾日，如易科罰金，以新臺幣壹仟元折算壹日。,臺灣桃園地方法院刑事判決106年度易字第1093號公訴人臺灣桃園地方法院檢察署檢察官被告邱建...,邱建豪於民國106年5月1日下午5時15分許，在桃園市○○區○○路000號桃園市政府警察局桃...
1,臺灣臺中地方法院,2011-05-19T00:00:00+08:00,"100,中交簡,1002",刑事,公共危險,判決,王炎權服用酒類，不能安全駕駛動力交通工具而駕駛，處拘役伍拾日，如易科罰金，以新臺幣壹仟元折算壹日。,臺灣臺中地方法院刑事簡易判決100年度中交簡字第1002號聲請人臺灣臺中地方法院檢察署檢察官...,王炎權於民國100年4月8日晚間9時30分許起至翌日凌晨3時許止，在彰化縣芬園鄉○○路○段2...


In [3]:
test=  pd.read_csv('.//data//Result_Test.csv', index_col=0)
test.head(2)

,court,date,no,sys,reason,type,mainText,judgement,Situation
id,,,,,,,,,
522,臺灣臺中地方法院,2017-02-06T00:00:00+08:00,"105,中簡,1134",刑事,傷害,判決,易佳騰犯傷害罪，處拘役拾日，如易科罰金，以新臺幣壹仟元折算壹日。,臺灣臺中地方法院刑事簡易判決105年度中簡字第1134號聲請人臺灣臺中地方法院檢察署檢察官被...,易佳騰於民國103年12月30日22時53分許，在臺中市○○區○○街000號，因工程款糾紛與...
901,臺灣臺中地方法院,2017-06-21T00:00:00+08:00,"105,原易,51",刑事,傷害,判決,宋約翰傷害人之身體，累犯，處有期徒刑肆月，如易科罰金，以新臺幣壹仟元折算壹日。,臺灣臺中地方法院刑事判決105年度原易字第51號公訴人臺灣臺中地方法院檢察署檢察官被告宋約翰...,宋約翰前於民國103年間因公共危險案件，經本院以103年度中交簡字第3997號判決判處有期徒...


In [4]:
df1 = df[~df['no'].isin(test['no'])]
df1['reason'] = df1['reason'].str.replace('等','')
df1['reason'] = df1['reason'].str.replace("條例","")
df1['reason'] = df1['reason'].str.replace("違反","")
#df1['Situation'] = df1['Situation'].str.replace(my_regex, "")
#ierror = 0
#for index, row in df1.iterrows():
#    if (row.Situation != row.Situation2):
#        print (index, "\n", row.Situation, "\n", row.Situation2)
#        ierror += 1
#        if (ierror > 5):
#            break

c:\users\user\pycharmprojects\pytorch\venv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\user\pycharmprojects\pytorch\venv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\user\pycharmprojects\pytorch\venv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [5]:
from sentence_transformers import SentenceTransformer, util
embedder=SentenceTransformer('bert-base-chinese')

import pandas as pd
import time
tStart = time.time()

Exception when trying to download https://sbert.net/models/bert-base-chinese.zip. Response 404


In [6]:
class Data(object):
    def __init__(self):
        self.no = "0"
        self.reason = ""
        self.Situation = ""

In [7]:
#embedder  SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Corpus with example sentences
#corpus  ['A man is eating food.',
#          'A man is eating a piece of bread.',
#          'The girl is carrying a baby.',
#          'A man is riding a horse.',
#          'A woman is playing violin.',
#          'Two men pushed carts through the woods.',
#          'A man is riding a white horse on an enclosed ground.',
#          'A monkey is playing drums.',
#          'A cheetah is running behind its prey.'
#          ]
corpus = []
corpusList = []
for index, row in df1.iterrows():
    if (row.Situation == None):
        continue
    if (isinstance(row.Situation, float)):
        continue
    if (len(row.Situation) == 0):
        continue
    if (index == 5):
        print()
    d = Data()
    d.no = row.no
    d.reason = row.reason
    d.Situation = row.Situation
    corpusList.append(d)
    corpus.append(row.Situation)
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
print(len(corpus))


3537


In [8]:
tEnd = time.time()#計時結束
#列印結果
print("轉向量時間", "%f sec" % (tEnd - tStart))

轉向量時間 23.629833 sec


In [9]:
# Query sentences:
#queries  ['A man is eating pasta.', 'Someone in a gorilla costume is playing a set of drums.', 'A cheetah chases prey on across a field.']
queries = []
specialtest  ='詎其猶不知悔改，復基於意圖為自己不法所有之竊盜犯意，於102年7月5日上午6時許（起訴書誤載為5時許），見林振億位於高雄市○○區○○街000號之住處大門未上鎖，竟未經允准侵入上開住宅內，徒手竊取行動電話5支、無線電3支、數位相機1臺及存錢筒1個（內有現金約新臺幣【下同】4,000元，上開物品共計價值47,000元），得手後離去；嗣經林振億發現遭竊後報警，為警在現場採得指紋後送鑑定後，結果與邱東成之指紋相符，而查悉上情。'
#specialtest = re.sub(my_regex, "", specialtest)
queries.append(specialtest)
for index, row in test.iterrows():
    queries.append(str(row.Situation))
print(len(queries))

22


In [10]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
i = 0
show = 0

for query in queries:
    tStart2 = time.time()
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()


    #We use np.argpartition, to only partially sort the top_k results
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    print("\n\n\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    no = 1
    for idx in top_results[0:top_k]:
        if (corpus[idx] == specialtest):
            continue
        print(no, "  "
              , corpusList[idx].no,"\t"
              ,  corpusList[idx].reason
              , "\t(Score: %.4f)" % (cos_scores[idx]))
        if (len(corpus[idx].strip()[0:100]) > 100):
            print (corpus[idx].strip()[0:100])
        else:
            print (corpus[idx].strip()[0:100])
        print('\n')
        no = no + 1
        show += 1
    if (show > 5):break

    tEnd2 = time.time()#計時結束
    print("COS相似計算時間", "%f sec" % (tEnd2 - tStart2))






Query: 詎其猶不知悔改，復基於意圖為自己不法所有之竊盜犯意，於102年7月5日上午6時許（起訴書誤載為5時許），見林振億位於高雄市○○區○○街000號之住處大門未上鎖，竟未經允准侵入上開住宅內，徒手竊取行動電話5支、無線電3支、數位相機1臺及存錢筒1個（內有現金約新臺幣【下同】4,000元，上開物品共計價值47,000元），得手後離去；嗣經林振億發現遭竊後報警，為警在現場採得指紋後送鑑定後，結果與邱東成之指紋相符，而查悉上情。

Top 5 most similar sentences in corpus:
1    105,審易,1055 	 竊盜 	(Score: 0.9833)
詎猶未悔改，復意圖為自己不法之所有，於100年3月18日17時30分前某時許，沿臺中市○○區○○路000號2樓之10林益宏住處陽臺外之鐵管攀爬進入陽臺後開啟未上鎖之窗戶，侵入該房屋內，徒手竊取林益宏所有之筆記型電腦1臺、撲滿1個【內約有現金新臺幣（下同）1000元至2000元之間】、手提包1只、數位相機與MP5各1臺（合計價值2萬9000元）等物，得手後除撲滿內之現金外，其餘財物均持往臺中火車站附近某不明舊貨攤商變賣，得款數千元，與撲滿內之現金均花用殆盡。嗣為警據報後，依其在上開房屋浴室內所留唾液跡證循線查獲。


2    102,審訴,269 	 搶奪 	(Score: 0.9831)
詎仍不知悔改，竟仍意圖為自己不法之所有，基於搶奪之犯意，於101年11月5日中午12時許，騎乘車牌號碼000-000號重型機車，行經桃園市○○○街00號新雅飯店前，見姚德慈獨自步行，認有機可乘，徒手搶奪姚德慈所有之手提包1個（內含有現金12,500元、行動電話1支、鑰匙1串、國民身分證1張、健保卡1張、汽車駕照1張、悠遊卡1張、中油加油卡1張、華南銀行信用卡1張、富邦銀行信用卡1張及提款卡2張等物，被害人均已領回），得手後旋即騎乘機車逃逸。嗣經警調閱監視器，循線於101年11月5日下午4時許，在桃園縣蘆竹鄉○○○路000號查獲，始悉上情。


3    105,審易,1268 	 竊盜 	(Score: 0.9829)
詎仍不知悔改，於99年12月27日凌晨3時許之夜間，行經陳幸慧位於臺中市○○區○○路00號住處前，發現該住處1樓落地鋁門窗未固定，認有機可趁，竟基於意圖為自己不法所有

In [11]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
embedder = SentenceTransformer('bert-base-chinese')
corpus_embeddings = embedder.encode(corpus)
# Then, we perform k-means clustering using sklearn:
from sklearn.cluster import KMeans

specialcls = -1

num_clusters = 18
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

Exception when trying to download https://sbert.net/models/bert-base-chinese.zip. Response 404


In [12]:
clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])
    if (specialtest == corpus[sentence_id]):
        specialcls = cluster_id
        print (cluster_id)
for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster[0].strip()[0:100])
    print("")

10
Cluster  1
被告蕭正吉基於賭博及意圖營利供給賭博場所、聚眾賭博之犯意，自民國99年12月間某日起，提供其位於臺中市○○區○○街184巷18號居處之公眾得出入之場所，聚集不特定多數人簽選號碼與之賭博財物，而經營「香

Cluster  2
詎其不知悔改，復基於賭博及意圖營利供給賭博場所、聚眾賭博之犯意，自民國100年1月25日起，提供其位在臺中市○○區○○路230號之公眾得出入之場所，聚集不特定多數人簽選號碼與之賭博財物。其賭法係賭客以

Cluster  3
王寬宏於民國100年2月16日上午9時55分許，駕駛車牌號碼TP-7158號自用小客車，沿臺南市○○區○○路由東往西方向行駛，行至復興路390號前時，本應注意汽車行駛時，駕駛人應注意車前狀況及兩車併行

Cluster  4
鍾信仁自民國100年5月初某日起，以每小時新臺幣（下同）200元之代價，受僱於真實姓名年籍均不詳綽號「安安」之成年女子所經營之應召站，擔任司機工作（即俗稱之車伕，並以其所有之門號0000000000號

Cluster  5
於九十九年十月六日某時許，在其位於臺南市○區○○街二段七九巷一四號之一之住處房間內，以將第一級毒品海洛因置入針筒內注射之方式，施用海洛因一次。另基於施用第二級毒品甲基安非他命之犯意，於同日某時許，在上

Cluster  6
於民國100年3月25日5時30分許，在臺中市南區永興5巷42號前，徒手竊取劉清智所有放置該處門口外之白鐵製水槽1個（價值約新臺幣「下同」2,000元），得手後，持往位於臺中市○里區○○○路1116巷

Cluster  7
被告柳慶揚意圖為自己不法之所有，於民國78年3月19日中午12時10分許，以治療牙痛為藉，至桃園縣楊梅鎮○○路000號東山牙科內就診，經謝亭龍以止痛藥予以止痛，並準備磨牙之際，竟取出預藏之水果刀1把，

Cluster  8
王炎權於民國100年4月8日晚間9時30分許起至翌日凌晨3時許止，在彰化縣芬園鄉○○路○段291巷15號之住處內飲酒後，已達不能安全駕駛動力交通工具之程度，竟仍駕駛車牌號碼為UJ-4775號自小客車上

Cluster  9
於99年12月15日中午12時許至同日23時45分前間之某時，在臺中市○○路140號前，以自備鑰匙插入電門啟動引擎後駛離之方式，竊取黃柑惠所有車牌號碼6L-7240

In [13]:
class Cls(object):
    def __init__(self):
        self.reason = ""
        self.num = 0

In [14]:
no = 1
clslist = {}
show = 0
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if (cluster_id != specialcls):
        continue
    if (show < 5):
        print(no, "  "
                  , corpusList[sentence_id].no,"\t"
                  ,  corpusList[sentence_id].reason, "\n")
        if (len(corpusList[sentence_id].Situation.strip()) > 100):
            print(corpusList[sentence_id].Situation.strip()[0:100])
        else:
            print(corpusList[sentence_id].Situation.strip())
        no += 1
        print("")

        show +=1

    if corpusList[sentence_id].reason in clslist:
        clslist[corpusList[sentence_id].reason] += 1
    else:
        clslist[corpusList[sentence_id].reason] = 1


1    100,中簡,1001 	 竊盜 

被告陳守仁為高職畢業擔任保全之男子，竟不思以正常手段獲取財物，因一時貪念，以徒手竊取該超商內商品供己食用之犯罪動機、目的、手段、生活狀況、智識程度、所竊得物品價值新臺幣（下同）總共約821元

2    100,中簡,1099 	 竊盜 

於九十九年十月二十八日上午九時三十分許，行經臺中市西屯區○○○街五九巷八號前，見張顯榮所有之車牌號碼ON－8455號自用小客車停放在該處，且無人看管，乃認有機可乘，竟意圖為自己不法之所有，持自備之機車

3    100,中簡,1257 	 竊盜 

於99年8月27日晚間11時50分許騎乘車牌號碼UAE-490號輕型機車，至臺中市○○區○○路與文昌東十二街口之兒童公園人行道，見李敏華所有之車牌號碼L6E-656號機車停放在該處，遂以不詳方式打開該

4    100,中簡,1358 	 竊盜 

於100年4月25日晚間10時39分許，經過臺中市○○區○○里○○街145巷1弄13號湯兆慶之住處時，見其住宅後門未關，遂基於意圖為自己不法所有之竊盜犯意，未經湯兆慶之同意，無故侵入湯兆慶前開住處內，

5    100,中簡,1385 	 竊盜 

於99年6月初某日，行經臺中市○區○○里○○○路○段77號前，見劉欣怡所有之THOMAS牌黑色可摺疊式6段變速腳踏車停放在該處，且未上鎖，竟意圖為自己不法之所有，基於竊盜之犯意，徒手行竊該部腳踏車（價



In [15]:
for key, value in clslist.items():
    print(key, " ", value)


竊盜   78
搶奪   124
強盜   76
妨害自由   5
傷害   1
毀損   4
妨害性自主   4
妨害風化   2
